This script is to compute FR at each cluster/supercluster for each timepoint

FMT dataset 2

In [1]:
# This script is to compute FR at each cluster/supercluster for each timepoint
# FMT dataset 2
import sys
sys.path.append('../src')
import os
import pandas as pd
import copy
import numpy as np
import tree_util


In [2]:
# compute FR for a sample
def fr_value(d_df, profile, sname):
    sp_list = list(profile.columns)
    n = len(sp_list)
    corr = np.ones(shape=(n, n)) - d_df.loc[sp_list, sp_list].values
    np.fill_diagonal(corr, 0)
    # print(corr[1, 1])
    a = np.array(profile.loc[sname, sp_list])
    inter_matrix = np.dot(a.reshape(len(a), 1),a.reshape(1, len(a)))
    np.fill_diagonal(inter_matrix, 0)
    td = np.sum(inter_matrix)/2
    fr = np.sum(np.multiply(inter_matrix, corr))/2
    fd = np.sum(np.multiply(inter_matrix, d_df.loc[sp_list, sp_list].values))/2
    if td == 0:
        return 0
    #return fr/td
    return fr

# compute FR for all sample
def multisample_fr(profile, d_df, node_leaves):
    result = pd.DataFrame(index=profile.index, columns=(list(node_leaves.keys()) + ['nroot']))
    for sname in profile.index:
       for node, sp_list in node_leaves.items():
           sp_list = list(set(sp_list).intersection(set(profile.columns)))
           selected_d = d_df.loc[sp_list, sp_list]
           selected_profile = profile.loc[:, sp_list]
           value = fr_value(selected_d, selected_profile, sname)
           result.loc[sname, node] = value
    return result


In [3]:
# load tree
with open('../result/GCN_fix_tree/renamed_GCN_tree.newick') as fp:
    newick_tree = fp.read()
    # newick_tree = newick_tree.replace('-', '_')

# construct tree with newick string
json_tree = tree_util.parse(newick_tree)
largest = {'largest': 0}
leaf_list, l = tree_util.recu_compute(json_tree, 0, largest)
largest_level = largest['largest']
nlayer = largest_level
layer_leaves_dict = tree_util.make_layer_dict(nlayer)

tree_util.recu_layer(json_tree, layer_leaves_dict)
tree_util.to_layer_leaves(layer_leaves_dict, nlayer)
leaves_dict = copy.deepcopy(layer_leaves_dict)

parent_dict = {}
tree_util.parents(json_tree, parent_dict)
node_leaves = {}
for level in layer_leaves_dict.keys():
    for node, sp_list in layer_leaves_dict[level].items():
        if node in node_leaves.keys():
            continue
        node_leaves[node] = copy.deepcopy(sp_list)
subtree_nodes = {}
for l in leaf_list:
    parent = parent_dict[l]
    if parent not in subtree_nodes.keys():
        subtree_nodes[parent] = []
    subtree_nodes[parent].append(l)

for node in node_leaves.keys():
    parent = parent_dict[node]
    if parent not in subtree_nodes.keys():
        subtree_nodes[parent] = []
    subtree_nodes[parent] += subtree_nodes[node]
    subtree_nodes[parent].append(node)

for node in subtree_nodes.keys():
    subtree_nodes[node].append(node)

direct_children_dict = {}
for node, parent in parent_dict.items():
    if parent not in direct_children_dict:
        direct_children_dict[parent] = []
    direct_children_dict[parent].append(node)

node_leaves['root'] = copy.deepcopy(leaf_list)

In [4]:
# load data
frac_df = pd.read_csv('../data/FMT/FMT2/Eric.txt', sep='\t', header=0, index_col=0)
deltat = pd.read_csv('../data/FMT/FMT2/deltat.txt', sep='\t', header=0, index_col=0)
triads = pd.read_csv('../data/FMT/FMT2/triads.txt', sep='\t', header=0, index_col=None)

In [5]:
for idx in triads.index:
    posts = triads.loc[idx, 'post_samples'].strip().split(',')
    times = []
    for p in posts:
        if p not in deltat.index:
            print(p)
        else:
            times.append(str(deltat.loc[p, 'delta_t']))
    t_str = ','.join(times)
    triads.loc[idx, 'days'] = t_str

In [6]:
for idx in frac_df.index:
    d = frac_df.loc[idx, 'Days postFMT']
    cand = triads[triads['days'] == d]['pre_sample']
    ids = triads[triads['days'] == d].index
    if len(cand) == 1:
        #print(list(cand))
        s = list(cand)[0]
        frac_df.loc[idx, 'sname'] = s
        triads.loc[list(ids)[0], 'alias'] = idx
triads.loc[10, 'alias'] = 'FMT5' 
s = triads.loc[10, 'pre_sample']
frac_df.loc['FMT5', 'sname'] = s

In [7]:
# load distance, abundance profile and metadata
outdir = '../result/FMT_FR/FMT2'
if not os.path.exists(outdir):
    os.makedirs(outdir)
d_df = pd.read_csv('../data/sp_d.tsv', sep='\t', header=0, index_col=0)
abd = pd.read_csv('../data/FMT/FMT2/Eric_abd.tsv', sep='\t', header=0, index_col=0)
name_dict = {}
for sp in d_df.columns:
        name_dict[sp] = sp.replace('_', '-')
d_df = d_df.rename(columns=name_dict, index=name_dict)
abd = abd.rename(columns=name_dict)
profile = abd[list(set(abd.columns).intersection(set(d_df.columns)))]
profile = profile.div(profile.sum(axis=1), axis=0)
profile = profile.rename(index={'FMT5':'FMT05', 'FMT6': 'FMT06'})


In [8]:
# compute FR for all samples at all timepoints
fr_df = multisample_fr(profile, d_df, node_leaves)
fr_df = fr_df.T
fr_df = fr_df.drop('nroot')
sample_name = '{}_day{}_{}'
triads_new = triads.set_index('pre_sample', inplace=False)
fr_df_result = pd.DataFrame(index=fr_df.index)
for idx in frac_df.index:
    ori_sname = frac_df.loc[idx, 'sname']
    new_sname = sample_name.format(idx, 0, ori_sname)
    days = triads_new.loc[ori_sname, 'days'].split(',')
    samples = triads_new.loc[ori_sname, 'post_samples'].split(',')
    fr_df_result[new_sname] = fr_df[ori_sname]
    for i, d in enumerate(days):
        fr_df_result[sample_name.format(idx, d, samples[i])] = fr_df[samples[i]]  
fr_df_result.to_csv(os.path.join(outdir, 'fr.tsv'), sep='\t')